## Viualizing

When creating structures, it's necessary to be shure 
that geometry of the system is correct, so there you could 
create 3D structures using functions from Geometry.


Please, before edditing this file run this command in shell
```bash
git update-index --skip-worktree Code/Rings_visualize.ipynb
```

It's necessary to avoid adding changes, based on 
calculations for different structures

If there is neccessary changes in open of save functions,
you could run this command in shell:

```bash
git update-index --no-skip-worktree Code/Rings_visualize.ipynb
```



In [6]:

import numpy as np
from Geometry import Rectangle_packing, Ellipse_packing
from Parameters import Params, Dz, Dy, Dx, Orientations
import plotly.graph_objects as go
import colormaps as cmaps
import matplotlib.pyplot as plt

# Choosing set of parameters


def plot_rings_plotly(rings:list, orientation:str, fig:go.Figure)->None:
    """Function to plot rings in 3D plotly figure

    Parameters
    ----------
    rings : list
        List of rings to plot
    orientation : str
        Orientation of the chosen rings
    fig : go.Figure
        Figure to plot the rings
    """    
    added_orientations = {'x': False, 'y': False, 'z': False}
    for ring in rings:
        x, y, z = [], [], []
        if ring.pos == orientation:
            theta = np.linspace(0, 2*np.pi, 100)
            dash = 'solid'
            width = 3
            alpha = 1
            if ring.pos == "x":
                x.extend(np.full((100,), ring.x))
                y.extend(ring.y + ring.r * np.cos(theta))
                z.extend(ring.z + ring.r * np.sin(theta))
                color = 'blue'
                name = 'X orientation' if not added_orientations['x'] else ''
                added_orientations['x'] = True if ring.R < 1e3 or added_orientations['x'] else False
            elif ring.pos == "y":
                x.extend(ring.x + ring.r * np.cos(theta))
                y.extend(np.full((100,), ring.y))
                z.extend(ring.z + ring.r * np.sin(theta))
                color = 'red'
                name = 'Y orientation' if not added_orientations['y'] else ''
                added_orientations['y'] = True if ring.R < 1e3 or added_orientations['y'] else False
            elif ring.pos == "z":
                x.extend(ring.x + ring.r * np.cos(theta))
                y.extend(ring.y + ring.r * np.sin(theta))
                z.extend(np.full((100,), ring.z))
                
                color = 'green'
                name = 'Z orientation' if not added_orientations['z'] else ''
                added_orientations['z'] = True if ring.R < 1e3 or added_orientations['z'] else False
            if ring.R >1e3:
                color = 'black' 
                width = 4
                dash = 'solid'
                alpha = 0.1
                name = ''
                
            fig.add_trace(go.Scatter3d(x=x, y=y, z=z,
                                       mode='lines',
                                       line=dict(color=color, width=width, dash=dash),
                                       name=name,
                                       opacity= alpha,
                                       showlegend=bool(name)))

def draw(rings, orientations):
    fig = go.Figure()
    for orientation in orientations:
        plot_rings_plotly(rings, orientation, fig)
    return fig

In [8]:
'''
Usage example
'''
# List of all rings
from Parameters import *
from Geometry import to3D, Packings


Params['Packing'] = 'Ellipse'
Type = 'border'
Params['N'], Params['Shape'] = to3D(5, 5, 5, 'zyx', Type=Type)
Rings_dict = Packings[Params['Packing']](Params, Fill = True)
Rings_list = np.concatenate([Rings_dict[pos] for pos in Rings_dict])

fig = draw(Rings_list, Orientations)
fig.update_layout(scene=dict(
    aspectmode='manual',  # Устанавливаем ручной режим
    aspectratio=dict(x=1, y=1, z=1),  # Устанавливаем одинаковое соотношение осей
))


# Remove axis ticks and labels
fig.update_layout(scene=dict(xaxis=dict(showticklabels=False, title=''),
                             yaxis=dict(showticklabels=False, title=''),
                             zaxis=dict(showticklabels=False, title='')))

# Setting background color as white
fig.update_layout(scene=dict(xaxis=dict(showbackground=False),
                             yaxis=dict(showbackground=False),
                             zaxis=dict(showbackground=False),
                             bgcolor='white'))
# Setting start view along z-axis
fig.update_layout(scene_camera=dict(eye=dict(x=0.15, y=2, z=0.15)))

# Saving frame to Plots/3D-structure
import os
if not os.path.exists("Images/3D-structure"):
    os.makedirs("Images/3D-structure")
fig.write_image(f"Images/3D-structure/{Params['Shape']}_{Params['Packing']}_{Type}.png",
                width=800, height=800, scale=2)
fig.show()
